In [35]:
print ('Hello')

Hello


In [0]:
#!pip install -U git+git://github.com/Computational-Content-Analysis-2020/lucem_illud_2020.git
import lucem_illud_2020 #pip install -U git+git://github.com/Computational-Content-Analysis-2020/lucem_illud_2020.git
import pandas #For DataFrames
import numpy as np #For arrays
import pickle #if you want to save layouts
import os
import re
import zipfile
import sys
import spacy
nlp=spacy.load('en_core_web_sm')

In [0]:
#final
def loadDavies(address, corpus_style="text", num_files=10000, return_raw=False):
    texts_raw = {}
    article_count=0
    for file in os.listdir(address + "/"):
        #change text name to month of interest
        if file.startswith('text_16-10-ykw.zip'):
            if corpus_style in file:
                print(type(file),file)
                zfile = zipfile.ZipFile(address + "/" + file)
                for file in zfile.namelist():
                    #check for us source
                    if file.lower().find('us')==-1:
                        continue
                    texts_raw[file] = []
                    with zfile.open(file) as f:
                        for article in f:
                            article_lower = article.decode().lower()
                            #only take articles with election in them
                            finde = 'election' in article_lower
                            if (finde):
                                article_count+=1
                                texts_raw[file].append(article)
    print (article_count)                            
    if return_raw:
        return(texts_raw)

In [38]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


In [39]:
#corpus_name = "/project2/soci40133/Davies_corpora/NOW"
corpus_name='/content/gdrive/My Drive/SOCI 40133 - Final Project/NOW/'
#news_raw_16_10 = loadDavies(corpus_name, corpus_style="text", num_files=10000, return_raw=True)
news_raw = loadDavies(corpus_name, corpus_style="text", num_files=10000, return_raw=True)


<class 'str'> text_16-10-ykw.zip
3343


In [0]:
#TODO:get rid of everything before the first p tag and store headline seperately
def clean_raw_text(raw_articles):
    clean_articles = []
    for text in raw_articles:
        try:
            text = text.decode("utf-8")
            clean_article = text.replace(" \'m", "'m").replace(" \'ll", "'ll").replace(" \'re", "'re").replace(" \'s", "'s").replace(" \'re", "'re").replace(" n\'t", "n't").replace(" \'ve", "'ve").replace(" /'d", "'d")\
                            .replace("<h>","").replace("<p>","").replace("@@","")
            clean_articles.append(clean_article)
        except AttributeError:
            # print("ERROR CLEANING")
            # print(text)
            continue
        except UnicodeDecodeError:
            # print("Unicode Error, Skip")
            continue
    return clean_articles

In [0]:
def word_tokenize(text, remove_stop_words=False):
    tokenized = []
    customize_stop_words = ['is']
    # pass word list through language model.
    doc = nlp(text)
    
    for w in customize_stop_words:
        nlp.vocab[w].is_stop = True
    for token in doc:
        if not token.is_punct and len(token.text.strip()) > 0:
            if remove_stop_words:
                if not token.is_stop: 
                   tokenized.append(token.text)
                else:
                    continue
            else:
                tokenized.append(token.text)
    
    return tokenized

In [0]:
import time

t0 = time.time()
ta = time.time()
a_count =0
news_texts = {}
just_texts = {}
for file in news_raw:
    print('File_name',file)
    #article=news_raw[file]
    articles = clean_raw_text(news_raw[file])
    article_cleaned=articles[0]
    for article in articles:
        a_count+=1
        if (a_count%100)==0:
            t=time.time()
            print(t-ta)
            ta=time.time()
            print('Processing',a_count)
        
        ##where we sentence break, tokenize sentences, flatten, etc.
        tokenized_sents = [lucem_illud_2020.word_tokenize(s) for s in lucem_illud_2020.sent_tokenize(article)]
        normalized_sents = [lucem_illud_2020.normalizeTokens(s) for s in tokenized_sents]
        #remove empty lists
        normalized_sents =  list(filter(None, normalized_sents))
        #print(normalized_sents)
        tokens = [item for sublist in normalized_sents for item in sublist]
        tid=article.split(" ")[0]
        #print(tid)
        #print(tokens)
        #tokens_spacy= word_tokenize(article)
        try:
            news_texts[tid] = (tokens,normalized_sents)
            just_texts[tid] = article
        except IndexError:
            print("Missing: ",txts[0][2:])
            continue
t1 = time.time()
print ("Total time", t1-t0)   
assert(len(list(news_texts.keys()))==a_count) 

File_name 16-10-us.txt
20.599775075912476
Processing 100
20.33160638809204
Processing 200
26.497512340545654
Processing 300
25.363152027130127
Processing 400
23.159037113189697
Processing 500
21.0468430519104
Processing 600
20.10713815689087
Processing 700
23.87464165687561
Processing 800
25.662846565246582
Processing 900
18.050594568252563
Processing 1000
25.70416760444641
Processing 1100
23.71855902671814
Processing 1200
20.77853035926819
Processing 1300
19.128224849700928
Processing 1400
19.510838270187378
Processing 1500
18.794106006622314
Processing 1600
17.715377807617188
Processing 1700
21.091196060180664
Processing 1800
21.04779028892517
Processing 1900
20.027034521102905
Processing 2000
19.35583472251892
Processing 2100
18.161215782165527
Processing 2200
19.860438108444214
Processing 2300
22.68381929397583
Processing 2400
20.206012725830078
Processing 2500
20.08288550376892
Processing 2600
19.89595365524292
Processing 2700
16.033664226531982
Processing 2800
18.87736439704895
P

In [0]:
print(article_cleaned)

14637217  Utility's rates , Old Market fire on minds of candidates for MUD board  Political newcomer Krystal Gabel says she's plenty comfortable being the only woman in a field of four contenders vying for two seats on the Metropolitan Utilities District board of directors .  Gabel , 31 , a technical writer for a local payments software company , said her relative youth and perspective would be an asset to the seven-member board .  " I have a new perspective on things , and while I don't want to play the gender card , there are a lot of men on the board , " she said . " When you lack equal representation , you're basically shutting out large populations from policymaking . "  Gabel also takes issue with what she says is the high fixed service charge on customers ' monthly bills . She said she wants to steer the utility toward increased transparency by showing ratepayers exactly what those charges fund within the district . Gabel also advocates more stringent water-quality standards and

In [0]:
ids=set((news_texts.keys()))

In [43]:
len(ids)

3343

In [44]:
# source matching
sources = []
try:
  zfile = zipfile.ZipFile(corpus_name + "/sources-16-10.zip")

  for file in zfile.namelist():
    with zfile.open(file) as f:
        for line in f:
            sources.append(line)
except:
  print('Source file not found')
  print('Using empty source')          

Source file not found
Using empty source


In [0]:
sl=[]
ecount=0
for s in sources:
    try:
        sl.append(s.decode().split("\t")[0].strip())
        #print(s.decode().split("\t")[2].strip())
    except UnicodeDecodeError:
        ecount+=1
        continue
    

In [47]:
print("Total source documents", len(sources))
print("Files with decode errors:", ecount)
print("Files whose sources intersect with the files we have collected", len(set(sl).intersection(ids)))
print ("Total files", len(ids))

Total source documents 0
Files with decode errors: 0
Files whose sources intersect with the files we have collected 0
Total files 3343


In [0]:
#news_texts[list(set(sl).intersection(ids))[0]]

In [0]:
news_df = pandas.DataFrame(columns=["title", "date", "country", "source", "url", "text", "tokens","normalized_tokens"])
count=0
sl = []
for news in sources:
    try:
        tid, total_words, date, country, source, url, title = news.decode("utf-8").split("\t")
    except UnicodeDecodeError:
        continue
    try:
        news_df.loc[tid.strip()] = [title.strip(), date.strip(), country.strip(), source.strip(), url.strip(), just_texts[tid.strip()], news_texts[tid.strip()][0],news_texts[tid.strip()][1]]
        sl.append(tid)
        count+=1
    except KeyError:
        continue
#the tids not found in sources
for tids in list(ids.difference(set(sl))):
    contents = (([""]+['16-10-00']))+['US']+ ([""]*2)+ [just_texts[tids], news_texts[tids][0],news_texts[tids][1]]
    #print ((contents))
    #print (type(contents))
    #try:
    news_df.loc[tids.strip()] = contents
    #except:
        #print (tids)

In [0]:
news_df

In [0]:
for i in range(news_df.shape[0]):
  if news_df['date']=='16-01-00'

SyntaxError: ignored

In [49]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [0]:
#sanity check
#news_df.loc['14637217']['tokens'][:100]

In [0]:
news_df.to_pickle('/content/drive/My Drive/2016-10_news_df.pkl')

In [53]:
df=pandas.read_pickle('/content/drive/My Drive/2016-10_news_df.pkl')
df

,title,date,country,source,url,text,tokens,normalized_tokens
14927162,,16-10-00,US,,,14927162 WASHINGTON -- A new poll has Republi...,"[washington, new, poll, republican, sen, rob, ...","[[washington], [new, poll, republican, sen, ro..."
14874872,,16-10-00,US,,,14874872 Top Story The first presidential de...,"[story, presidential, debate, probably, cheer,...","[[story], [presidential, debate, probably, che..."
14771204,,16-10-00,US,,,14771204 After a video surfaced of Donald Tru...,"[video, surface, donald, trump, make, vulgar, ...","[[video, surface, donald, trump, make, vulgar,..."
14819341,,16-10-00,US,,,14819341 Presidential election cycles are alw...,"[presidential, election, cycle, grind, year, e...","[[presidential, election, cycle, grind], [year..."
15010047,,16-10-00,US,,,15010047 After what seemed like a sleepy star...,"[like, sleepy, start, saturday, college, footb...","[[like, sleepy, start, saturday, college, foot..."
...,...,...,...,...,...,...,...,...
14833865,,16-10-00,US,,,14833865 Four women claim Trump touched them ...,"[woman, claim, trump, touch, inappropriately, ...","[[woman, claim, trump, touch, inappropriately,..."
15034592,,16-10-00,US,,,15034592 Testing Gary Johnson claim that he i...,"[test, gary, johnson, claim, candidate, favor,...","[[test, gary, johnson, claim, candidate, favor..."
14921871,,16-10-00,US,,,14921871 What we've learned so far from the h...,"[have, learn, far, hack, email, hillary, clint...","[[have, learn, far, hack, email, hillary, clin..."
15066814,,16-10-00,US,,,15066814 Amid recent shuffles in the kitchen ...,"[amid, recent, shuffle, kitchen, ned, ludd, co...","[[amid, recent, shuffle, kitchen, ned, ludd, c..."


In [0]:
type(df.loc['14761029']['normalized_tokens'])

list